# Presentation of the LSTM ANN model
...

In [40]:
# module imports
import os
import sys
import math
import datetime
from datetime import date
import itertools
import time as t
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.tsa import stattools
from tabulate import tabulate

import math
import keras as keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
from keras.callbacks import TensorBoard
from keras.utils import np_utils
from keras.models import load_model

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from IPython.display import HTML
from IPython.display import display

import ipywidgets as widgets

# Import custom module functions
module_path = os.path.abspath(os.path.join(''))
if module_path not in sys.path:
    sys.path.append(module_path)
from lstm_load_forecasting import data, lstm

from ast import literal_eval

from dateutil.tz import tzutc

import pytz

modules = widgets.ToggleButtons(
    options={'ENTSO-E FC Only':['entsoe'], 'Calendar Only':['calendar'], 'Weather only':['weather'], 'Calendar + Weather':['calendar', 'weather'], 'All':['all']},
    description='Choose data:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Choose the data module that the model should be run with',
)
update_date = widgets.DatePicker(description='Update date')
display(modules, update_date)
path = os.path.join(os.path.abspath(''), 'Data/fulldataset.csv')

sel_mod = modules.value
sel_mod.append('actual')

In [41]:
df = data.load_dataset(path=path, modules=['all'], update_date = update_date.value)

### Options 

In [44]:
timezone = pytz.timezone('Europe/Zurich')
starting = timezone.localize(datetime.datetime(2017,5,15,0,0,0,0))
path = os.path.join('Data', 'fulldataset.csv')
res_path = os.path.abspath('results/')
model_dir = os.path.abspath('models/')
date = '20170508'

# Data preparation

In [45]:
df3 = df
df3_scaled = df3.copy()
df3_scaled = df3_scaled.dropna()

# Get all float type columns
floats = [key for key in dict(df3_scaled.dtypes) if dict(df3_scaled.dtypes)[key] in ['float64']]
scaler = StandardScaler()
scaled_columns = scaler.fit_transform(df3_scaled[floats])
df3_scaled[floats] = scaled_columns

df3_train = df3_scaled.loc[(df3_scaled.index < starting )].copy()
df3_test = df3_scaled.loc[df3_scaled.index >= starting].copy()
y_train = df3_train['actual'].copy()
X_train = df3_train.drop('actual', 1).copy()
y_test = df3_test['actual'].copy()
X_test = df3_test.drop('actual', 1).copy()

valid_results_3 = pd.read_csv(os.path.join(res_path, 'notebook_03/', str('model3_results_' + date + '.csv')), delimiter=';')
test_results_3 = pd.read_csv(os.path.join(res_path, 'notebook_03/', str('model3_test_results' + date + '.csv')), delimiter=';')
test_results_3 = test_results_3.sort_values('Mean absolute error', ascending=True)
best_model_3 = test_results_3.loc[0]['Model name']

config = valid_results_3.loc[valid_results_3['model_name'] == best_model_3]
batch_size = int(config['batch_train'].values[0])
size = int(y_test.shape[0] / batch_size)

layers = literal_eval(config['config'].values[0])
layers = layers['layers']

# Create and Train model

In [46]:
batch_size = 8
model3 = lstm.create_model(layers=layers, sample_size=X_train.shape[0], batch_size=batch_size, timesteps=1, 
                           features=X_train.shape[1], loss='mse', optimizer='adam')
history = lstm.train_model(model=model3, mode='fit', y=y_train, X=X_train, 
                           batch_size=batch_size, timesteps=1, epochs=25, 
                           rearrange=False, validation_split=0.2, verbose=1)

Warnining: Division "sample_size/batch_size" not a natural number.
Dropped the last 7 of 20711 number of obs.
Effective validation split now is: 0.200
Train on 16568 samples, validate on 4136 samples
Epoch 1/25
16568/16568 [==============================] - 11s - loss: 0.5338 - mean_absolute_error: 0.4890 - val_loss: 0.1999 - val_mean_absolute_error: 0.3639
Epoch 2/25
16568/16568 [==============================] - 10s - loss: 0.4044 - mean_absolute_error: 0.3975 - val_loss: 0.1748 - val_mean_absolute_error: 0.3390
Epoch 3/25
16568/16568 [==============================] - 10s - loss: 0.3823 - mean_absolute_error: 0.3754 - val_loss: 0.1836 - val_mean_absolute_error: 0.3502
Epoch 4/25
16568/16568 [==============================] - 10s - loss: 0.3663 - mean_absolute_error: 0.3644 - val_loss: 0.1998 - val_mean_absolute_error: 0.3674
Epoch 5/25
16568/16568 [==============================] - 10s - loss: 0.3596 - mean_absolute_error: 0.3590 - val_loss: 0.2073 - val_mean_absolute_error: 0.3774


In [55]:
starting_loc = starting
starting_utc = starting.astimezone(pytz.utc)
ending_utc = df3_test.tail(1).index[0]
ending_loc = ending_utc.tz_convert('Europe/Zurich')
day_changes = pd.date_range(start=starting_loc, end=ending_loc, freq='24h', tz="Europe/Zurich")

predictions = pd.DataFrame(index=pd.date_range(start=starting_loc, end=ending_loc, normalize=True, freq='60min', tz='Europe/Zurich'))
predictions.index = predictions.index.tz_convert('utc')

for idx, hour in enumerate(day_changes):
    if idx == 0:
        pass
    else:
        dh = df3_test.loc[(df3_test.index >= hour - pd.DateOffset(days=1)) & (df3_test.index < hour)]
        y_train = dh['actual']
        X_train = dh.drop('actual', 1)
        history = lstm.train_model(model=model3, mode='fit', y=y_train, X=X_train, 
                                   batch_size=batch_size, timesteps=1, epochs=100, rearrange=False, validation_split=0,
                                   verbose=0, early_stopping=False)
    
    df = df3_test.loc[(df3_test.index >= hour) & (df3_test.index < hour + pd.DateOffset(days=2))]
    df = df.drop('actual', 1)
    X_predict = df
    scaled_predictions = lstm.get_predictions(model=model3, X=X_predict, batch_size=batch_size, timesteps=1, verbose=1)
    hour_utc = hour.tz_convert('utc')
    window = pd.date_range(start=hour_utc, periods=len(scaled_predictions), freq='60min', tz='UTC')
    result = pd.DataFrame(data={"model3": scaled_predictions.flatten()}, index=window)
    predictions = predictions.combine_first(result)

mu = scaler.mean_[0]
sigma = scaler.scale_[0]
model3_predictions = mu + sigma*predictions

 8/24 [=========>....................] - ETA: 0s

In [56]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
dfc = data.load_dataset(path=path, modules=['actual', 'entsoe'])
forecasts = model3_predictions
forecasts = forecasts.join(dfc)
forecasts = forecasts.dropna()
results = {}
results[''] = ['MSE', 'MAE', 'MAPE']
results['entsoe'] = [mean_squared_error(forecasts['actual'], forecasts['entsoe']), 
                     mean_absolute_error(forecasts['actual'], forecasts['entsoe']),
                     mean_absolute_percentage_error(forecasts['actual'], forecasts['entsoe'])
                    ]
results['m3-calendar'] = [mean_squared_error(forecasts['actual'], forecasts['model3']), 
                          mean_absolute_error(forecasts['actual'], forecasts['model3']),
                          mean_absolute_percentage_error(forecasts['actual'], forecasts['model3'])
                         ]

print(tabulate(results, headers='keys', numalign="right", tablefmt='grid', floatfmt=".3f"))

+------+-----------+---------------+
|      |    entsoe |   m3-calendar |
+======+===========+===============+
| MSE  | 14018.062 |     84382.357 |
+------+-----------+---------------+
| MAE  |    77.729 |       229.974 |
+------+-----------+---------------+
| MAPE |     1.100 |         3.395 |
+------+-----------+---------------+


In [57]:
%matplotlib
plt.figure()
plt.plot(forecasts.index, forecasts['entsoe'], label='ENTSOE Forecast')
plt.plot(forecasts.index, forecasts['actual'], label='Actual Load')
plt.plot(forecasts.index, forecasts['model3'], label='Model 3 (Calendar)')
plt.title('Forecast Comparison: Rolling Forecast')
plt.ylabel('Electricity load (in MW)')
plt.xlabel('Date')
plt.legend(loc='upper left')
plt.show

Using matplotlib backend: TkAgg


<function matplotlib.pyplot.show>

In [ ]:
K.clear_session()
import tensorflow as tf
tf.reset_default_graph()